<a href="https://colab.research.google.com/github/mateopolancecofficial/NLP/blob/main/WineReviews/SequenceModelText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pip install -q -U tensorflow-text

In [3]:
pip install -q -U tf-models-official

In [4]:
#pip install -U tfds-nightly

In [50]:
!git clone -l -s https://github.com/mateopolancecofficial/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 52 (delta 20), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [4]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np

tf.get_logger().setLevel('ERROR')

In [5]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [6]:
import os

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

Using TPU


In [7]:
df = pd.read_csv("/content/NLP/WineReviews/data/winemag-data-130k-v2.csv", index_col=0)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [8]:
df = df[['description', 'points']]
df.head()

,description,points
0,"Aromas include tropical fruit, broom, brimston...",87
1,"This is ripe and fruity, a wine that is smooth...",87
2,"Tart and snappy, the flavors of lime flesh and...",87
3,"Pineapple rind, lemon pith and orange blossom ...",87
4,"Much like the regular bottling from 2012, this...",87


In [9]:
df = df.sample(frac = 1)
df.head()

,description,points
56537,Here's a pleasantly fragrant blend of Negroama...,87
492,"Tart and lively, this medium-bodied Sangiovese...",87
112903,The latest release from Vallado is a value win...,87
127846,"The wine has good acidity, crisp green and whi...",87
84888,"The wine is dry, although its soft and creamy ...",88


In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt



In [11]:
df.description = df.description.str.replace('\d+', '')

In [12]:
from sklearn.model_selection import train_test_split

text_train, text_val, y_train, y_val = train_test_split(df.description, df.points,
                                                  test_size=0.2,train_size=0.8)

In [42]:
tf.config.run_functions_eagerly(False)

def create_model():
  hub_model = "https://tfhub.dev/google/nnlm-en-dim50/2"
  hub_layer = hub.KerasLayer(hub_model, input_shape=[], dtype=tf.string, trainable=True)
  model = tf.keras.Sequential()
  model.add(hub_layer)
  #model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(8, activation='relu'))
  #model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(1))
  return model


In [92]:

#with strategy.scope():
from tensorflow.keras import backend as K

def root_mean_squared_log_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y_true))))

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

loss = 'root_mean_squared_error'
metric = 'mean_absolute_error'

model = create_model()

model.compile(optimizer='rmsprop',
              loss=root_mean_squared_error,
              metrics=[metric])
  

Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7f81646e3320>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/p

In [87]:
checkpoint_path = "/content/NLP/WineReviews/checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 1024

AUTOTUNE = tf.data.AUTOTUNE
seed = 42

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

train_dataset = tf.data.Dataset.from_tensor_slices((text_train, y_train))
train_dataset = train_dataset.batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((text_val, y_val))
val_dataset = val_dataset.batch(batch_size)

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [23]:
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=1,
                    )

1625/1625 [==============================] - 20s 13ms/step - loss: 58.5678 - mean_squared_error: 58.5678 - val_loss: 42.2012 - val_mean_squared_error: 42.2012


In [24]:
model.save_weights(checkpoint_path.format(epoch=0))

In [26]:


model.save_weights('./saved_weights.h5', overwrite=True)

# Load the previously saved weights
#model.load_weights(latest)

In [29]:
model.load_weights("./saved_weights.h5")

In [30]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    )



Epoch 1/10
1625/1625 [==============================] - 20s 12ms/step - loss: 30.0545 - mean_squared_error: 30.0545 - val_loss: 18.7864 - val_mean_squared_error: 18.7864
Epoch 2/10
1625/1625 [==============================] - 20s 12ms/step - loss: 12.7159 - mean_squared_error: 12.7159 - val_loss: 8.6578 - val_mean_squared_error: 8.6578
Epoch 3/10
1625/1625 [==============================] - 21s 13ms/step - loss: 7.5965 - mean_squared_error: 7.5965 - val_loss: 7.0713 - val_mean_squared_error: 7.0713
Epoch 4/10
1625/1625 [==============================] - 20s 12ms/step - loss: 7.0580 - mean_squared_error: 7.0580 - val_loss: 6.9815 - val_mean_squared_error: 6.9815
Epoch 5/10
1625/1625 [==============================] - 20s 12ms/step - loss: 7.0333 - mean_squared_error: 7.0333 - val_loss: 6.9779 - val_mean_squared_error: 6.9779
Epoch 6/10
1625/1625 [==============================] - 20s 12ms/step - loss: 7.0322 - mean_squared_error: 7.0322 - val_loss: 6.9778 - val_mean_squared_error: 6.977

In [36]:
model.save_weights('./weights.h5', overwrite=True)

In [37]:
model.load_weights("./weights.h5")

In [38]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    callbacks=[es_callback]
                    )

Epoch 1/10
1625/1625 [==============================] - 21s 13ms/step - loss: 7.0171 - mean_squared_error: 7.0171 - val_loss: 6.9526 - val_mean_squared_error: 6.9526
Epoch 2/10
1625/1625 [==============================] - 21s 13ms/step - loss: 7.0070 - mean_squared_error: 7.0070 - val_loss: 6.9421 - val_mean_squared_error: 6.9421
Epoch 3/10
1625/1625 [==============================] - 21s 13ms/step - loss: 6.9977 - mean_squared_error: 6.9977 - val_loss: 6.9338 - val_mean_squared_error: 6.9338
Epoch 4/10
1625/1625 [==============================] - 20s 12ms/step - loss: 6.9901 - mean_squared_error: 6.9901 - val_loss: 6.9266 - val_mean_squared_error: 6.9266
Epoch 5/10
1625/1625 [==============================] - 21s 13ms/step - loss: 6.9843 - mean_squared_error: 6.9843 - val_loss: 6.9208 - val_mean_squared_error: 6.9208
Epoch 6/10
1625/1625 [==============================] - 20s 12ms/step - loss: 6.9795 - mean_squared_error: 6.9795 - val_loss: 6.9160 - val_mean_squared_error: 6.9160
Epoc

In [40]:
model.save_weights('./weights.h5', overwrite=True)

In [44]:
model.load_weights("./weights.h5")

In [47]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    callbacks=[es_callback]
                    )

Epoch 1/10
102/102 [==============================] - 18s 174ms/step - loss: 3.2101 - mean_squared_error: 3.2101 - val_loss: 3.2121 - val_mean_squared_error: 3.2121
Epoch 2/10
102/102 [==============================] - 18s 176ms/step - loss: 2.8417 - mean_squared_error: 2.8417 - val_loss: 3.0564 - val_mean_squared_error: 3.0564
Epoch 3/10
102/102 [==============================] - 17s 168ms/step - loss: 2.6186 - mean_squared_error: 2.6186 - val_loss: 2.9699 - val_mean_squared_error: 2.9699
Epoch 4/10
102/102 [==============================] - 17s 170ms/step - loss: 2.4623 - mean_squared_error: 2.4623 - val_loss: 2.9230 - val_mean_squared_error: 2.9230
Epoch 5/10
102/102 [==============================] - 17s 169ms/step - loss: 2.3472 - mean_squared_error: 2.3472 - val_loss: 2.9006 - val_mean_squared_error: 2.9006
Epoch 6/10
102/102 [==============================] - 17s 170ms/step - loss: 2.2593 - mean_squared_error: 2.2593 - val_loss: 2.8945 - val_mean_squared_error: 2.8945
Epoch 7/10

In [48]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    callbacks=[es_callback]
                    )

Epoch 1/10
102/102 [==============================] - 18s 173ms/step - loss: 2.0151 - mean_squared_error: 2.0151 - val_loss: 2.9574 - val_mean_squared_error: 2.9574
Epoch 2/10
102/102 [==============================] - 18s 172ms/step - loss: 1.9842 - mean_squared_error: 1.9842 - val_loss: 2.9757 - val_mean_squared_error: 2.9757
Epoch 3/10
102/102 [==============================] - 18s 173ms/step - loss: 1.9561 - mean_squared_error: 1.9561 - val_loss: 2.9934 - val_mean_squared_error: 2.9934
Epoch 4/10
102/102 [==============================] - 18s 176ms/step - loss: 1.9298 - mean_squared_error: 1.9298 - val_loss: 3.0113 - val_mean_squared_error: 3.0113
Epoch 5/10
102/102 [==============================] - 18s 176ms/step - loss: 1.9053 - mean_squared_error: 1.9053 - val_loss: 3.0302 - val_mean_squared_error: 3.0302


In [72]:
df = pd.read_csv("/content/NLP/WineReviews/data/winemag-data-130k-v2.csv", index_col=0)
df = df[['description', 'points']]
df = df.sample(frac = 1)
df.description = df.description.str.replace('\d+', '')

df_1 = pd.read_csv("/content/NLP/WineReviews/data/winemag-data_first150k.csv", index_col=0)
df_1 = df_1[['description', 'points']]
df_1 = df_1.sample(frac = 1)
df_1.description = df_1.description.str.replace('\d+', '')

In [73]:
df_all = pd.concat([df_1, df])

In [74]:
df_all.shape

(280901, 2)

In [75]:
# dropping duplicte values
df_all.drop_duplicates(subset ="description",
                       keep = 'first', inplace = True)

In [89]:
df_all.shape

(169429, 2)

In [90]:
text_train, text_val, y_train, y_val = train_test_split(df_all.description, 
                                                        df_all.points,
                                                  test_size=0.2,train_size=0.8)
batch_size = 1024

AUTOTUNE = tf.data.AUTOTUNE
seed = 42

train_dataset = tf.data.Dataset.from_tensor_slices((text_train, y_train))
train_dataset = train_dataset.batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((text_val, y_val))
val_dataset = val_dataset.batch(batch_size)

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [91]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=30,
                    callbacks=[es_callback]
                    )

Epoch 1/30


TypeError: ignored

In [63]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    callbacks=[es_callback]
                    )

Epoch 1/10
77/77 [==============================] - 14s 173ms/step - loss: 2.0286 - mean_squared_error: 2.0286 - val_loss: 2.9920 - val_mean_squared_error: 2.9920
Epoch 2/10
77/77 [==============================] - 13s 173ms/step - loss: 1.9767 - mean_squared_error: 1.9767 - val_loss: 3.0384 - val_mean_squared_error: 3.0384
Epoch 3/10
77/77 [==============================] - 14s 176ms/step - loss: 1.9321 - mean_squared_error: 1.9321 - val_loss: 3.0846 - val_mean_squared_error: 3.0846
Epoch 4/10
77/77 [==============================] - 13s 174ms/step - loss: 1.8931 - mean_squared_error: 1.8931 - val_loss: 3.1284 - val_mean_squared_error: 3.1284
Epoch 5/10
77/77 [==============================] - 13s 175ms/step - loss: 1.8579 - mean_squared_error: 1.8579 - val_loss: 3.1709 - val_mean_squared_error: 3.1709


In [67]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    callbacks=[es_callback]
                    )

Epoch 1/10
77/77 [==============================] - 14s 178ms/step - loss: 7508.3454 - mean_absolute_error: 86.5792 - val_loss: 6260.2783 - val_mean_absolute_error: 79.0686
Epoch 2/10
77/77 [==============================] - 14s 186ms/step - loss: 5342.7893 - mean_absolute_error: 72.7148 - val_loss: 2028.3466 - val_mean_absolute_error: 44.7291
Epoch 3/10
77/77 [==============================] - 15s 189ms/step - loss: 1229.1385 - mean_absolute_error: 32.9244 - val_loss: 129.4379 - val_mean_absolute_error: 8.9864
Epoch 4/10
77/77 [==============================] - 14s 180ms/step - loss: 117.7783 - mean_absolute_error: 8.5160 - val_loss: 106.4393 - val_mean_absolute_error: 8.1043
Epoch 5/10
77/77 [==============================] - 14s 184ms/step - loss: 104.1191 - mean_absolute_error: 7.9779 - val_loss: 98.5340 - val_mean_absolute_error: 7.7890
Epoch 6/10
77/77 [==============================] - 14s 185ms/step - loss: 95.8042 - mean_absolute_error: 7.6456 - val_loss: 91.8382 - val_mean_ab

In [85]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=30,
                    callbacks=[es_callback]
                    )

Epoch 1/30


ValueError: ignored

In [88]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=30,
                    callbacks=[es_callback]
                    )

Epoch 1/30


TypeError: ignored

From the Colli Fiorentini zone of Chianti, this beautiful vin santo spent five years maturing in small oak barrels. It shows flavors of candied fruit, honey, resin, pine nut and toasted almond.                                                                                         1
This is the winery's first Petit Verdot, and it's a beauty. Firm and muscular, it has plenty of blackberry and black cherry flavors hanging on its tannic bones. The finish shows a smack of dark chocolate, and the overall structure suggests that this will age for  years or more.    1
With an opulent texture, this is ripe and packed with yellow fruits. There is just a hint of the wood aging along with great acidity and citrus flavors. Although full-bodied, the wine finishes crisply fresh.                                                                           1
As always, a nice, easy-drinking Chard with plenty of Central Coast character. The cool, long hangtime has given very ripe fruit, while preserving v